# Execute the Graph-Based Spatial Cross-Validation experiments from ICMLA21 Paper

## 1 - Initialize libraries

In [1]:
import os
import pandas as pd
import geopandas as gpd
from weka.core import jvm
from pathlib import Path
from src import utils
from src.pipeline import Pipeline
#from src.visualization.performance import VizMetrics
#from src.visualization.dependence import VizDependence

## 2 - Initialize loggers

In [2]:
utils.initialize_coloredlog()
utils.initialize_rich_tracerback()
utils.initialize_logging()

## 3 - Initialize working path and enviromental variables

In [3]:
# Project path
project_dir = str(Path().resolve().parents[1])
# Load enviromental variables
env_var = utils.load_env_variables(project_dir)
# Load parameters
dataset = "NSW_Election_2019"
parameters = utils.load_json(os.path.join(project_dir, "parameters", "validation", f"{dataset}.json"))

## 4 - Set pipeline switchers, the default is to set True to all processes

In [4]:
# Set pipeline switchers


SWITCHERS = {
    "scv": False,
    "fs": False,
    "train": False,
    "predict": True,
    "evaluate": True,
}

# 4 - List all datasets

In [5]:
dataset_path = os.path.join(env_var["root_path"], dataset)
dataset_list = [folder for folder in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, folder))]
dataset_list.remove("Original")
dataset_list.sort()
dataset_list = ["Original"]

## 6 - Runs the pipeline for each method
OBS: The results and files generated from the pipeline execution will be in the created folder Results in the data directory

### 6.1 Traditional SCV
OBS: We the the paramenter fast True so the semivariogram calculation step that can take 24h is skipped. We calculate the removing buffer by considering the 27 n-degree neighborhood as stated in the paper 

In [6]:
   
fs_method = parameters["fs_method"]
ml_methods = parameters["ml_methods"]
# Set paths
meshblocks_filename = parameters["meshblock"]
meshblocks_idx = parameters["meshblock_id"]
dataset_path = os.path.join(env_var["root_path"], dataset)

if fs_method == "CFS" and SWITCHERS["fs"]:
    jvm.start()
for dataset in dataset_list:
    data_path = os.path.join(dataset_path, dataset, "data.csv")
    meshblocks_path = os.path.join(dataset_path, dataset, "meshblocks", meshblocks_filename)
    # Load data
    data = pd.read_csv(data_path, index_col=parameters["index_col"], low_memory=False)
    meshblocks = gpd.read_file(meshblocks_path)
    # Remove uncessary cols
    if parameters["cols_remove"]:
        try:
            data.drop(columns=parameters["cols_remove"], inplace=True)
        except KeyError:
            pass
    # Set meshblocks  index
    if meshblocks_idx:
        meshblocks.set_index(meshblocks_idx, inplace=True)
    
    for ml_method in ml_methods:
        # Run pipeline
        TraditionalSCV = Pipeline(
            root_path=os.path.join(dataset_path, dataset),
            data=data,
            meshblocks=meshblocks,
            index_col=parameters["index_col"],
            fold_col=parameters["fold_col"],
            target_col=parameters["target_col"],
            scv_method="TraditionalSCV",
            fs_method=fs_method,
            ml_method=ml_method,
            switchers=SWITCHERS
        )

        print(f"Running the Traditional SCV approach for dataset: {dataset} ML Method = {ml_method}")
        TraditionalSCV.run()
if fs_method == "CFS" and SWITCHERS["fs"]:
    jvm.stop()

Running the Traditional SCV approach for dataset: Original ML Method = SVM


Evaluating predictions: 100%|██████████| 47/47 [02:34<00:00,  3.28s/it]
